## Feature selection by using anova test

In [1]:
# Read the final version file of dataset after prepocess
import pandas as pd
df = pd.read_csv("../data/curated/price_normalised3.csv")
df = df.rename(columns={'train_station(m)':'train_station','CBD(m)':'CBD'}) 

## Using ANOVA helps to determine if a categorical variable has influence on a numerical variable

#### link: https://towardsdatascience.com/statistics-in-python-using-anova-for-feature-selection-b4dc876ef4f0#:~:text=ANOVA%20is%20used%20for%20testing%20two%20variables%2C%20where%3A,categorical%20variable%202%20another%20is%20a%20numerical%20variable


if p < 0.05, this means that the categorical variable has significant influence on the numerical variable
if p > 0.05, this means that the categorical variable has no significant influence on the numerical variable

## ANOVA

In [2]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('cost_text ~ C(property_type) + C(bedrooms) + C(bathrooms) + C(parking) + schools + train_station + CBD + population2022 + income2022',
            data=df).fit()
df_feature=sm.stats.anova_lm(model, typ=2)
df_feature

,sum_sq,df,F,PR(>F)
C(property_type),4.004823e+06,14.0,6.637327,1.354623e-13
C(bedrooms),2.539828e+07,11.0,53.573437,4.080512e-114
C(bathrooms),3.151468e+07,9.0,81.247174,1.217086e-143
C(parking),2.815069e+06,13.0,5.024393,6.448718e-09
schools,1.110345e+06,1.0,25.762971,3.964761e-07
train_station,1.017019e+07,1.0,235.975557,2.320107e-52
CBD,2.228292e+07,1.0,517.023168,2.689856e-110
population2022,4.878717e+06,1.0,113.199226,3.164387e-26
income2022,7.001461e+05,1.0,16.245256,5.626814e-05
Residual,2.874239e+08,6669.0,NaN,NaN


From above tavle, it can be found that the p-vale of all variables are less than 0.05, which means all variables have significant influence on rental price

In [3]:
# save ouput to csv file
df_feature.to_csv('../data/curated/feature_selection.csv')

### Useful internal features: property_type, bedrooms, bathrooms, parking
### Useful external features: distance to the nearest school, distance to the nearest train_station, distance to CBD, population, median income

### Assumption: since bedrooms, bathrooms, parking are ordinal variables, so we asume they are categorical variables
### Limtation: Anova determines with numerical variables(distance to the nearest school&train station&CBD) may not very accurately 